In [1]:
import numpy as np
from neural_networks import NeuralNet
import scipy.special as sp

In [15]:
#Data load
X_train = np.load('/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/post_pca_data/X_train.npy')
Y_train = np.load('/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/post_pca_data/Y_train.npy')


In [3]:
class NeuralNet(object):
    RNG = np.random.default_rng()

    def __init__(self, topology: list[int] = []):
        self.topology = topology
        self.weight_mats = []

        self._init_matrices()

    def _init_matrices(self):
        # Set up weight matrices
        if len(self.topology) > 1:
            j = 1
            for i in range(len(self.topology) - 1):
                num_rows = self.topology[i]
                num_cols = self.topology[j]

                mat = self.RNG.random(size=(num_rows, num_cols))
                self.weight_mats.append(mat)

                j += 1

    def feedforward(self, input_vector):
        I = input_vector

        for idx, W in enumerate(self.weight_mats):
            I = np.dot(I, W)

            if idx == len(self.weight_mats) - 1:
                out_vector = sp.softmax(I)  # Output layer
            else:
                I = sp.softmax(I)  # Hidden layers

        return out_vector


In [16]:
#Define topology based on data's feature size and desired layers
input_size = X_train.shape[1]  # Number of features in your data
nnet = NeuralNet(topology=[input_size, 4, 5])  # Example topology: input -> 4 hidden -> 2 output

In [17]:
# Perform feedforward on the first sample
output = nnet.feedforward(data[500])  # Pass the first data point to the network
print(output)

[0.25239578 0.13368343 0.27491265 0.15515806 0.18385007]


In [18]:
#Train the model
def cross_entropy_loss(predictions, true_labels):
    return -np.sum(true_labels * np.log(predictions + 1e-9)) / true_labels.shape[0]


In [19]:
def train(neural_net, X_train, y_train, learning_rate, epochs):
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y_true in zip(X_train, y_train):
            # Forward pass
            y_pred = neural_net.feedforward(x)
            
            # Compute loss
            loss = cross_entropy_loss(y_pred, y_true)
            epoch_loss += loss
            
            # Backpropagation (compute gradients) - this part needs to be implemented
            
            # Update weights - modify neural_net.weight_mats using the gradients
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


In [20]:
def train(neural_net, X_train, y_train, learning_rate, epochs):
    def softmax_derivative(output):
        # Softmax derivative for backpropagation
        return output * (1 - output)
    
    for epoch in range(epochs):
        epoch_loss = 0
        
        for x, y_true in zip(X_train, y_train):
            # Forward pass
            activations = [x]  # Store activations for backpropagation
            current_input = x

            # Forward propagate through layers
            for W in neural_net.weight_mats:
                current_input = sp.softmax(np.dot(current_input, W))  # Activation
                activations.append(current_input)

            y_pred = activations[-1]

            # Compute loss
            loss = cross_entropy_loss(y_pred, y_true)
            epoch_loss += loss

            # Backpropagation
            deltas = []  # Store deltas for each layer
            delta = y_pred - y_true  # Output layer delta
            deltas.append(delta)

            # Compute gradients for each layer (backpropagate)
            for i in range(len(neural_net.weight_mats) - 1, 0, -1):
                delta = (deltas[-1] @ neural_net.weight_mats[i].T) * softmax_derivative(activations[i])
                deltas.append(delta)

            deltas.reverse()  # Reverse deltas to match layers

            # Update weights
            for i in range(len(neural_net.weight_mats)):
                grad = np.outer(activations[i], deltas[i])  # Gradient of the weights
                neural_net.weight_mats[i] -= learning_rate * grad  # Gradient descent step
        
        # Print epoch loss
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


In [22]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_train_one_hot = encoder.fit_transform(Y_train.reshape(-1, 1))


/Users/yasminebenmessaoud/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [23]:
# Define training parameters
learning_rate = 0.01
epochs = 100

# Train the model
train(nnet, data, y_train_one_hot, learning_rate, epochs)


Epoch 1/100, Loss: 0.27406637717677235
Epoch 2/100, Loss: 0.23794535597408056
Epoch 3/100, Loss: 0.2280596629223684
Epoch 4/100, Loss: 0.232774355825564
Epoch 5/100, Loss: 0.21815980990848075
Epoch 6/100, Loss: 0.2159402507150379
Epoch 7/100, Loss: 0.2436841584113011
Epoch 8/100, Loss: 0.2352916807778647
Epoch 9/100, Loss: 0.23108824613824158
Epoch 10/100, Loss: 0.22984334563326542
Epoch 11/100, Loss: 0.21396638220415565
Epoch 12/100, Loss: 0.22098355020664298
Epoch 13/100, Loss: 0.2395154016790717
Epoch 14/100, Loss: 0.2398165478839683
Epoch 15/100, Loss: 0.21986515584993738
Epoch 16/100, Loss: 0.24335434047388585
Epoch 17/100, Loss: 0.2423404261014221
Epoch 18/100, Loss: 0.24194774602544933
Epoch 19/100, Loss: 0.241868409942294
Epoch 20/100, Loss: 0.24148876038590028
Epoch 21/100, Loss: 0.24103943359166444
Epoch 22/100, Loss: 0.24092297554396055
Epoch 23/100, Loss: 0.2410491700568421
Epoch 24/100, Loss: 0.24101024230412949
Epoch 25/100, Loss: 0.2409059890969257
Epoch 26/100, Loss: 0.

In [24]:
# Predict for a new data sample (e.g., the first row of your dataset)
new_sample = data[0]
prediction = nnet.feedforward(new_sample)

# Convert the output (softmax probabilities) to a predicted class
predicted_class = np.argmax(prediction)
print(f"Predicted Class: {predicted_class}")


Predicted Class: 0


In [34]:
import numpy as np
import scipy.special as sp
from sklearn.preprocessing import OneHotEncoder


# Neural Network Class
class NeuralNet:
    RNG = np.random.default_rng()

    def __init__(self, topology: list[int]):
        self.topology = topology
        self.weight_mats = []
        self._init_matrices()

    def _init_matrices(self):
        for i in range(len(self.topology) - 1):
            num_rows = self.topology[i]
            num_cols = self.topology[i + 1]
            mat = self.RNG.random(size=(num_rows, num_cols))
            self.weight_mats.append(mat)

    def feedforward(self, input_vector):
        I = input_vector
        for idx, W in enumerate(self.weight_mats):
            I = np.dot(I, W)
            I = sp.softmax(I)  # Softmax activation for all layers
        return I


# Cross-Entropy Loss Function
def cross_entropy_loss(predictions, true_labels):
    return -np.sum(true_labels * np.log(predictions + 1e-9)) / predictions.shape[0]


# Training Function
def train(neural_net, X_train, y_train, learning_rate, epochs):
    def softmax_derivative(output):
        return output * (1 - output)

    for epoch in range(epochs):
        epoch_loss = 0
        for x, y_true in zip(X_train, y_train):
            # Forward pass
            activations = [x]
            current_input = x
            for W in neural_net.weight_mats:
                current_input = sp.softmax(np.dot(current_input, W))
                activations.append(current_input)
            y_pred = activations[-1]

            # Compute loss
            loss = cross_entropy_loss(y_pred, y_true)
            epoch_loss += loss

            # Backpropagation
            deltas = [y_pred - y_true]  # Output layer delta
            for i in range(len(neural_net.weight_mats) - 1, 0, -1):
                delta = (deltas[-1] @ neural_net.weight_mats[i].T) * softmax_derivative(activations[i])
                deltas.append(delta)
            deltas.reverse()

            # Update weights
            for i in range(len(neural_net.weight_mats)):
                grad = np.outer(activations[i], deltas[i])
                neural_net.weight_mats[i] -= learning_rate * grad

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


# Main Code
if __name__ == "__main__":
    # Load data
    X_train = np.load('/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/post_pca_data/X_train.npy')
    Y_train = np.load('/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/post_pca_data/Y_train.npy')

    # One-hot encode labels
    encoder = OneHotEncoder(sparse=False)
    y_train_one_hot = encoder.fit_transform(Y_train.reshape(-1, 1))

    # Define topology and initialize the neural network
    input_size = X_train.shape[1]
    output_size = y_train_one_hot.shape[1]
    nnet = NeuralNet(topology=[input_size, 4, output_size])

    # Train the network
    learning_rate = 0.01
    epochs = 100
    train(nnet, X_train, y_train_one_hot, learning_rate, epochs)

    # Predict for a new data sample
    new_sample = X_train[0]
    prediction = nnet.feedforward(new_sample)
    predicted_class = np.argmax(prediction)
    print(f"Predicted Class: {predicted_class}")


/Users/yasminebenmessaoud/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100, Loss: 0.2669152627695626
Epoch 2/100, Loss: 0.25190917336565805
Epoch 3/100, Loss: 0.252434698686746
Epoch 4/100, Loss: 0.25234939448211796
Epoch 5/100, Loss: 0.23997267820060747
Epoch 6/100, Loss: 0.24954406126360848
Epoch 7/100, Loss: 0.2449900080665397
Epoch 8/100, Loss: 0.24430162550259363
Epoch 9/100, Loss: 0.23857706390767927
Epoch 10/100, Loss: 0.23074394152804875
Epoch 11/100, Loss: 0.2209596013242533
Epoch 12/100, Loss: 0.2174964798461392
Epoch 13/100, Loss: 0.213188401787977
Epoch 14/100, Loss: 0.2090858072049219
Epoch 15/100, Loss: 0.20784727312270676
Epoch 16/100, Loss: 0.20776043106385517
Epoch 17/100, Loss: 0.20756249517441516
Epoch 18/100, Loss: 0.21109727131947978
Epoch 19/100, Loss: 0.21233324515155513
Epoch 20/100, Loss: 0.21507219659798357
Epoch 21/100, Loss: 0.20916804988658763
Epoch 22/100, Loss: 0.20764136670357694
Epoch 23/100, Loss: 0.2081594047954068
Epoch 24/100, Loss: 0.20793572727590479
Epoch 25/100, Loss: 0.21625346197635173
Epoch 26/100, Loss:

In [33]:
# Train the network
learning_rate = 0.01
epochs = 100
train(nnet, X_train, y_train_one_hot, learning_rate, epochs)

# Predict for a new data sample
new_sample = X_train[56]
prediction = nnet.feedforward(new_sample)
predicted_class = np.argmax(prediction)
print(f"Predicted Class: {predicted_class}")

Epoch 1/100, Loss: 0.20239656275051804
Epoch 2/100, Loss: 0.20208265865284472
Epoch 3/100, Loss: 0.20205921811411065
Epoch 4/100, Loss: 0.20204962323976894
Epoch 5/100, Loss: 0.2020442636064452
Epoch 6/100, Loss: 0.20204094826481703
Epoch 7/100, Loss: 0.20203879662669122
Epoch 8/100, Loss: 0.2020373588149479
Epoch 9/100, Loss: 0.20203637671334362
Epoch 10/100, Loss: 0.2020356931080051
Epoch 11/100, Loss: 0.20203520893098878
Epoch 12/100, Loss: 0.20203486033791637
Epoch 13/100, Loss: 0.20203460545853555
Epoch 14/100, Loss: 0.2020344164018954
Epoch 15/100, Loss: 0.20203427430632023
Epoch 16/100, Loss: 0.2020341662216964
Epoch 17/100, Loss: 0.2020340831200053
Epoch 18/100, Loss: 0.2020340186110971
Epoch 19/100, Loss: 0.20203396810429705
Epoch 20/100, Loss: 0.20203392825459826
Epoch 21/100, Loss: 0.20203389659220333
Epoch 22/100, Loss: 0.2020338712713858
Epoch 23/100, Loss: 0.20203385089779935
Epoch 24/100, Loss: 0.2020338344080082
Epoch 25/100, Loss: 0.20203382098419018
Epoch 26/100, Loss